In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data Reading 

import os
from glob import glob
from PIL import Image

# Data Processing 

import numpy as np
import pandas as pd
import cv2
import random
import albumentations as A

# Data Analysis

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Data Modeling & Model Evaluation

from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import layers, models
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow.keras.preprocessing import image

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, recall_score, accuracy_score, precision_score, f1_score, roc_auc_score

# Grad-CAM

import keras
import matplotlib.cm as cm

#Shap
import shap

#Sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling  import RandomUnderSampler 

# Data Gathering

In [ ]:
levels = ['NORMAL', 'COVID19', 'PNEUMONIA']
train_path = "../input/chest-xray-covid19-pneumonia/Data/train"
test_path = "../input/chest-xray-covid19-pneumonia/Data/test"
train_data_dir = os.path.join(train_path)
test_path_dir = os.path.join(test_path)

train_data = []
for id, level in enumerate(levels):
    for file in os.listdir(os.path.join(train_data_dir, level)):
        train_data.append(['{}/{}'.format(level, file), level])

train_data = pd.DataFrame(train_data, columns = ['image_file', 'corona_result']) 
train_data['path'] = train_path + '/' + train_data['image_file']
              
test_data = []
for id, level in enumerate(levels):
    for file in os.listdir(os.path.join(test_path_dir, level)):
        test_data.append(['{}/{}'.format(level, file), level])
        
test_data = pd.DataFrame(test_data, columns = ['image_file', 'corona_result'])
test_data['path'] = test_path + '/' + test_data['image_file']


train_data['corona_result'] = train_data['corona_result'].map({'NORMAL': 'NORMAL', 'COVID19': 'COVID19', 'PNEUMONIA': 'PNEUMONIA'})
test_data['corona_result'] = test_data['corona_result'].map({'NORMAL': 'NORMAL', 'COVID19': 'COVID19', 'PNEUMONIA': 'PNEUMONIA'})
samples = 5144

data = []
data = train_data
data.head()

In [ ]:
data['image'] = data['path'].map(lambda x: np.asarray(Image.open(x).resize((75,75))))

data.head()

**List for images and their labels + Histogram Equalisation**

In [ ]:
all_data = []

# Storing images and their labels into a list for further Train Test split

for i in range(len(data)):
    image = cv2.imread(data['path'][i])
    
    #histogram equalisation
    image=cv2.equalizeHist(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    image = np.repeat(image[..., np.newaxis], 3, -1)
    
    image = cv2.resize(image, (70, 70)) / 255.0
    if data['corona_result'][i] == "COVID19" :
        label = 1
    elif data['corona_result'][i] == "NORMAL" :
        label = 0
    else:
        label = 2
    all_data.append([image, label])

**Train-Test Split : Without Resampling**

In [ ]:
x = []
y = []

for image, label in all_data:
    x.append(image)
    y.append(label)

    
# Converting to Numpy Array    
x = np.array(x)
y = np.array(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 42)


print(x_train.shape, x_test.shape, x_val.shape, y_train.shape, y_test.shape, y_val.shape)

**Train-Test Split : With Oversampling** using SMOTE

In [ ]:
x = []
y = []

for image, label in all_data:
    x.append(image)
    y.append(label)

# Converting to Numpy Array    
x = np.array(x)
y = np.array(y)
print(x.shape)

dataForSmote = x.reshape(5144,70 * 70*3)
print(dataForSmote.shape)

sm = SMOTE(random_state=42)
X_smote, y_smote = sm.fit_resample(dataForSmote, y)
new_X = X_smote.reshape(-1,70,70,3)
print(new_X.shape)
print(y_smote.shape)

unique, counts = np.unique(y_smote, return_counts=True)
print(dict(zip(unique, counts)))

x_ovtrain, x_ovtest, y_ovtrain, y_ovtest = train_test_split(new_X, y_smote, test_size = 0.2, random_state = 42)
x_ovtrain, x_ovval, y_ovtrain, y_ovval = train_test_split(x_ovtrain, y_ovtrain, test_size = 0.1, random_state = 42)


print(x_ovtrain.shape, x_ovtest.shape, x_ovval.shape, y_ovtrain.shape, y_ovtest.shape, y_ovval.shape)

**Train-Test Split : With Undersampling** using Random Undersampler

In [ ]:
x = []
y = []

for image, label in all_data:
    x.append(image)
    y.append(label)

# Converting to Numpy Array    
x = np.array(x)
y = np.array(y)
print(x.shape)


dataForrus = x.reshape(5144,70 * 70 * 3)
#print(dataForrus.shape)

 
rus = RandomUnderSampler(random_state=10)
X_rus, y_rus = rus.fit_resample(dataForrus, y)
new_X = X_rus.reshape(-1,70,70,3)
print(new_X.shape)
#print(y_rus[14])

unique, counts = np.unique(y_rus, return_counts=True)
print(dict(zip(unique, counts)))

x_rustrain, x_rustest, y_rustrain, y_rustest = train_test_split(new_X, y_rus, test_size = 0.2, random_state = 42)
x_rustrain, x_rusval, y_rustrain, y_rusval = train_test_split(x_rustrain, y_rustrain, test_size = 0.1, random_state = 42)


print(x_rustrain.shape, x_rustest.shape, x_rusval.shape, y_rustrain.shape, y_rustest.shape, y_rusval.shape)

**Build DenseNet121 Model**

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import (
    Dense, Conv2D, MaxPool2D, Dropout, Flatten, 
    BatchNormalization, GlobalAveragePooling2D
)
from keras.models import Sequential, Model

In [ ]:
# model = Sequential()

base_model = DenseNet121(input_shape=(70,70,3), include_top=False, weights='imagenet')

layers = base_model.layers

x = base_model.output

x = GlobalAveragePooling2D()(x)

predictions = Dense(3, activation="softmax")(x)

model_densenet = Model(inputs=base_model.input, outputs=predictions)

model_densenet.summary()

**Compile DenseNet121 Model**

In [ ]:
model_densenet.compile(optimizer = 'adam', 
                    loss = 'sparse_categorical_crossentropy', 
                    metrics = ['accuracy'])

**Fitting without Resampling**

In [ ]:
es = keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 4)

model_densenet.fit(x_train,y_train,
          epochs=10,
          batch_size=256,
          validation_data=(x_val,y_val),
          callbacks = [es])

**Fitting with Oversampling**

In [ ]:
es = keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 4)

model_densenet.fit(x_ovtrain,y_ovtrain,
          epochs=10,
          batch_size=256,
          validation_data=(x_ovval,y_ovval),
          callbacks = [es])

**Fitiing with Undersampling**

In [ ]:
es = keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 4)

model_densenet.fit(x_rustrain,y_rustrain,
          epochs=10,
          batch_size=256,
          validation_data=(x_rusval,y_rusval),
          callbacks = [es])

**ROC-AUC_SCORE**

In [ ]:
#To obtain RUC AUC Score for without resampling method

yp_test = model_densenet.predict(x_test)

auc = roc_auc_score(y_test, yp_test,average = 'macro', multi_class="ovr")
print('ROC AUC: %f' % auc)

In [ ]:
#To obtain RUC AUC Score for Oversampling method

yp_test = model_densenet.predict(x_ovtest)

auc = roc_auc_score(y_ovtest, yp_test,average = 'macro', multi_class="ovr")
print('ROC AUC: %f' % auc)

In [ ]:
#To obtain RUC AUC Score for Undersampling method

yp_test = model_densenet.predict(x_rustest)

auc = roc_auc_score(y_rustest, yp_test,average = 'macro', multi_class="ovr")
print('ROC AUC: %f' % auc)

**EVALUATION PARAMETERS**

In [ ]:
yp_train = model_densenet.predict(x_train)
yp_train = np.argmax(yp_train, axis = 1)

yp_val = model_densenet.predict(x_val)
yp_val = np.argmax(yp_val, axis = 1)

yp_test = model_densenet.predict(x_test)
yp_test = np.argmax(yp_test, axis = 1)

In [ ]:
yp_train = model_densenet.predict(x_ovtrain)
yp_train = np.argmax(yp_train, axis = 1)

yp_val = model_densenet.predict(x_ovval)
yp_val = np.argmax(yp_val, axis = 1)

yp_test = model_densenet.predict(x_ovtest)
yp_test = np.argmax(yp_test, axis = 1)

In [ ]:
yp_train = model_densenet.predict(x_rustrain)
yp_train = np.argmax(yp_train, axis = 1)

yp_val = model_densenet.predict(x_rusval)
yp_val = np.argmax(yp_val, axis = 1)

yp_test = model_densenet.predict(x_rustest)
yp_test = np.argmax(yp_test, axis = 1)

In [ ]:
def evaluation_parametrics(name, y_train, yp_train, y_val, yp_val, y_test, yp_test):
    
    print("\n-----------------------------{}-----------------------------\n".format(name))
    
    cm_train = confusion_matrix(y_train, yp_train)
    t1 = ConfusionMatrixDisplay(cm_train)
    s1 = round((cm_train[0,0]/(cm_train[0,0] + cm_train[0,1])),4)
    
    print("Classification Report for Train Data\n")
    print(classification_report(y_train, yp_train)) 
    print("--------------------------------------------------------------------------")
    print("Recall on Train Data: ", round(recall_score(y_train, yp_train,average='micro'),4))
    print("Specificity on Train Data: ", s1)
    print("Accuracy on Train Data: ", round(accuracy_score(y_train, yp_train),4))
    print("Precision on Train Data: ", round(precision_score(y_train, yp_train,average='micro'),4))
    print("F1 Score on Train Data: ", round(f1_score(y_train, yp_train,average='micro'),4))
    print("--------------------------------------------------------------------------")
       
    cm_val = confusion_matrix(y_val, yp_val)
    t2 = ConfusionMatrixDisplay(cm_val)
    s2 = round((cm_val[0,0]/(cm_val[0,0] + cm_val[0,1])),4)
    
    print("\nClassification Report for Validation Data\n")
    print(classification_report(y_val, yp_val))   
    print("--------------------------------------------------------------------------")
    print("Recall on Val Data: ", round(recall_score(y_val, yp_val,average='micro'),4))
    print("Specificity on Val Data: ", s2)
    print("Accuracy on Val Data: ", round(accuracy_score(y_val, yp_val),4))
    print("Precision on Val Data: ", round(precision_score(y_val, yp_val,average='micro'),4))
    print("F1 Score on Val Data: ", round(f1_score(y_val, yp_val,average='micro'),4))
    print("--------------------------------------------------------------------------")

    cm_test = confusion_matrix(y_test, yp_test)
    t3 = ConfusionMatrixDisplay(cm_test)
    s3 = round((cm_test[0,0]/(cm_test[0,0] + cm_test[0,1])),4)
    
    print("\nClassification Report for Test Data\n")
    print(classification_report(y_test, yp_test))   
    print("--------------------------------------------------------------------------")
    print("Recall on Test Data: ", round(recall_score(y_test, yp_test,average='micro'), 4))
    print("Specificity on Test Data: ", s3)
    print("Accuracy on Test Data: ", round(accuracy_score(y_test, yp_test), 4))
    print("Precision on Test Data: ", round(precision_score(y_test, yp_test,average='micro'), 4))
    print("F1 Score Test Data: ", round(f1_score(y_test, yp_test,average='micro'), 4))
    print("--------------------------------------------------------------------------")
    
    t1.plot()
    t2.plot()   
    t3.plot()

***Metrics - Without Resampling***

In [ ]:
evaluation_parametrics("DenseNet121", y_train, yp_train, y_val, yp_val, y_test, yp_test)

***Metrics - With Oversampling***

In [ ]:
evaluation_parametrics("DenseNet121", y_ovtrain, yp_train, y_ovval, yp_val, y_ovtest, yp_test)

***Metrics - With Undersampling***

In [ ]:
evaluation_parametrics("DenseNet121", y_rustrain, yp_train, y_rusval, yp_val, y_rustest, yp_test)

# SHAP VALUES

**Predicted values from the model are used to determine SHAP values, labelling multi-class images into different dictionary for computing predictions and then analysing the SHAP value results.**

* y_train, y_test - evaluation parameter for without resampling
* y_ovtrain, y_ovtest - evaluation parameter for Oversampling
* y_rustrain, y_rustest - evaluation parameter for Undersampling

In [ ]:
print(np.where(y_train == 0))
print(np.where(y_train == 1))
print(np.where(y_train == 2))

In [ ]:
print(np.where(y_test == 0))
print(np.where(y_test == 1))
print(np.where(y_test == 2))

In [ ]:
print(np.where(y_ovtrain == 0))
print(np.where(y_ovtrain == 1))
print(np.where(y_ovtrain == 2))

In [ ]:
print(np.where(y_ovtest == 0))
print(np.where(y_ovtest == 1))
print(np.where(y_ovtest == 2))

In [ ]:
print(np.where(y_rustrain == 0))
print(np.where(y_rustrain == 1))
print(np.where(y_rustrain == 2))

In [ ]:
print(np.where(y_rustest == 0))
print(np.where(y_rustest == 1))
print(np.where(y_rustest == 2))

In [ ]:
class_labels= ["0","1","2"]
# example image for each class
images_dict = dict()
images_dict[0]= x_train[0]
images_dict[1]= x_train[4]
images_dict[2]= x_train[2]

images_dict = dict(sorted(images_dict.items()))
# example image for each class for test set
x_test_dict = dict()
x_test_dict[0]= x_test[4]
x_test_dict[1]= x_test[19]
x_test_dict[2]= x_test[0] 

# order by class
x_test_each_class = [x_test_dict[i] for i in sorted(x_test_dict)]
x_test_each_class = np.asarray(x_test_each_class)

# Compute predictions
predictions = model_densenet.predict(x_test_each_class)
predicted_class = np.argmax(predictions, axis=1)

In [ ]:
class_labels= ["0","1","2"]
# example image for each class
images_dict = dict()
images_dict[0]= x_ovtrain[0]
images_dict[1]= x_ovtrain[1]
images_dict[2]= x_ovtrain[2]

images_dict = dict(sorted(images_dict.items()))
# example image for each class for test set
x_test_dict = dict()
x_test_dict[0]= x_ovtest[12]
x_test_dict[1]= x_ovtest[7]
x_test_dict[2]= x_ovtest[3]

# order by class
x_test_each_class = [x_test_dict[i] for i in sorted(x_test_dict)]
x_test_each_class = np.asarray(x_test_each_class)

# Compute predictions
predictions = model_densenet.predict(x_test_each_class)
predicted_class = np.argmax(predictions, axis=1)

In [ ]:
class_labels= ["0","1","2"]
# example image for each class
images_dict = dict()
images_dict[0]= x_rustrain[24]
images_dict[1]= x_rustrain[15]
images_dict[2]= x_rustrain[0]

images_dict = dict(sorted(images_dict.items()))
# example image for each class for test set
x_test_dict = dict()
x_test_dict[0]= x_rustest[275]
x_test_dict[1]= x_rustest[1]
x_test_dict[2]= x_rustest[2]

# order by class
x_test_each_class = [x_test_dict[i] for i in sorted(x_test_dict)]
x_test_each_class = np.asarray(x_test_each_class)

# Compute predictions
predictions = model_densenet.predict(x_test_each_class)
predicted_class = np.argmax(predictions, axis=1)

# Generate the SHAP values:

In [ ]:
# select a set of background examples to take an expectation over

background = x_train[np.random.choice(x_train.shape[0], 100, replace=False)]

explainer = shap.DeepExplainer(model_densenet, background)

shap_val = explainer.shap_values(x_test_each_class,ranked_outputs=None)

In [ ]:
# select a set of background examples to take an expectation over

background = x_ovtrain[np.random.choice(x_ovtrain.shape[0], 100, replace=False)]

explainer = shap.DeepExplainer(model_densenet, background)

shap_val = explainer.shap_values(x_test_each_class,ranked_outputs=None)

In [ ]:
# select a set of background examples to take an expectation over

background = x_rustrain[np.random.choice(x_rustrain.shape[0], 100, replace=False)]

explainer = shap.DeepExplainer(model_densenet, background)

shap_val = explainer.shap_values(x_test_each_class,ranked_outputs=None)

In [ ]:
# plot actual and predicted class
def plot_actual_predicted(images, pred_classes):
  fig, axes = plt.subplots(1, 4, figsize=(6, 5))
  axes = axes.flatten()
  
  # plot
  ax = axes[0]
  dummy_array = np.array([[[0, 0, 0, 0]]], dtype='uint8')
  ax.set_title("Base reference")
  ax.set_axis_off()
  ax.imshow(dummy_array, interpolation='nearest')
  # plot image
  for k,v in images.items():
    ax = axes[k+1]
    ax.imshow(v, cmap=plt.cm.binary)
    ax.set_title(f"True: %s \nPredict: %s" % (class_labels[k], class_labels[pred_classes[k]]))
    ax.set_axis_off()
  plt.tight_layout()
  plt.show()

***Visualization of the SHAP values using image_plot***

Red pixels represent positive SHAP values that contributed to classifying that image as that particular class.


Blue pixels represent negative SHAP values that contributed to not classifying that image as that particular class.

In [ ]:
plot_actual_predicted(x_test_dict,predicted_class)
shap.image_plot(shap_val,x_test_each_class)